In [23]:
from matplotlib import pyplot as plt
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import matplotlib.colors as mcolors

def visualize_graph_from_file(file_path, metrics_to_plot, statistics_to_plot):
    """
    Visualizza un grafo 3D interattivo a partire da un file CSV.

    :param file_path: Il percorso del file CSV contenente i dati.
    :param metrics_to_plot: Lista delle metriche da visualizzare (ad esempio ['closeness', 'degree']).
    :param statistics_to_plot: Lista delle statistiche da visualizzare (ad esempio ['Mean', 'Median']).
    """
    # Leggi il file CSV
    data = pd.read_csv(file_path)

    # Estrai nodi unici e metriche uniche
    nodes = data['Node'].unique()
    metrics = data['Metric'].unique()

    # Funzione per creare il grafo e assegnare le metriche come attributi dei nodi
    def create_graph_with_metrics(data):
        G = nx.Graph()

        for node in nodes:
            node_data = data[data['Node'] == node]
            node_metrics = {
                row['Metric']: {
                    'Mean': row['Mean'],
                    'Median': row['Median'],
                    'StdDev': row['Standard Deviation']
                } for _, row in node_data.iterrows()
            }
            G.add_node(node, **node_metrics)

        return G

    # Crea il grafo con le metriche
    G = create_graph_with_metrics(data)

    # Funzione per visualizzare il grafo interattivo con Plotly
    def visualize_graph_by_statistic_interactive(G, metric, statistic, title="Graph Visualization"):
        """
        Visualizza un grafo in 3D con Plotly, colorando i nodi in base alla statistica selezionata.
        """
        pos = nx.spring_layout(G, dim=3, seed=42)

        # Estrai i valori della statistica per colorare i nodi
        values = [G.nodes[node].get(metric, {}).get(statistic, 0) for node in G.nodes()]
        colormap = plt.cm.plasma  # Usa una colormap vivace
        norm = mcolors.Normalize(vmin=min(values), vmax=max(values))

        # Ottieni le posizioni dei nodi
        x = [pos[i][0] for i in G.nodes()]
        y = [pos[i][1] for i in G.nodes()]
        z = [pos[i][2] for i in G.nodes()]

        # Colori per i nodi
        colors = [colormap(norm(value)) for value in values]

        # Crea il grafico 3D interattivo con Plotly
        trace_nodes = go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers+text',
            text=[str(node) for node in G.nodes()],
            textposition="top center",
            marker=dict(
                size=10,
                color=values,
                colorscale='plasma',
                colorbar=dict(title=f'{metric} ({statistic})')
            ),
            name=f'Nodes ({metric} - {statistic})'
        )

        # Crea gli spigoli del grafo
        edge_x = []
        edge_y = []
        edge_z = []
        for edge in G.edges():
            x0, y0, z0 = pos[edge[0]]
            x1, y1, z1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])

        trace_edges = go.Scatter3d(
            x=edge_x, y=edge_y, z=edge_z,
            mode='lines',
            line=dict(color='black', width=2),
            name='Edges'
        )

        # Layout del grafico
        layout = go.Layout(
            title=title,
            scene=dict(
                xaxis=dict(title='X'),
                yaxis=dict(title='Y'),
                zaxis=dict(title='Z')
            ),
            showlegend=True
        )

        # Crea la figura e visualizzala
        fig = go.Figure(data=[trace_edges, trace_nodes], layout=layout)
        fig.show()

    # Visualizza il grafo per ogni combinazione di metrica e statistica selezionata
    for metric in metrics_to_plot:
        for statistic in statistics_to_plot:
            visualize_graph_by_statistic_interactive(G, metric=metric, statistic=statistic,
                                                     title=f"Graph Colored by {metric.capitalize()} - {statistic}")


In [24]:
file_path = 'analysis/ppmi/60_70/comparison/pd/node_differences.csv'
metrics_to_plot = ['closeness', 'degree']  # Le metriche che vuoi visualizzare
statistics_to_plot = ['Mean', 'Median']   # Le statistiche che vuoi visualizzare

# Visualizza il grafo con i parametri selezionati
visualize_graph_from_file(file_path, metrics_to_plot, statistics_to_plot)
